## Data Cleaning and Preprocessing

In [1]:
# import all relevant libraries 

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score, make_scorer
from yellowbrick.cluster import KElbowVisualizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from rapidfuzz import process, fuzz
from sklearn.cluster import AgglomerativeClustering
from sklearn.manifold import TSNE
from scipy.sparse import hstack


In [2]:
main_df = pd.read_csv("nlrb_current_final_6-20.csv")

# find all columns with more than 80% missing data
missing_threshold = 0.8
columns_with_missing_data = main_df.columns[main_df.isnull().mean() > missing_threshold]

print(f"Columns with more than 80% missing data: {list(columns_with_missing_data)}")

Columns with more than 80% missing data: ['label', 'ent_road', 'ent_house_number', 'ent_unit', 'ent_level', 'union', 'lab_entity', 'lab_house_number', 'lab_road', 'lab_city', 'lab_state', 'lab_postcode', 'lab_unit', 'lab_level', 'lab_phone_number']


In [3]:
threshold = 0.8 

# drop columns with more than 80% of missing data in the col
main_df = main_df.loc[:, main_df.isnull().mean() <= threshold]

In [4]:
new_order = [
            'entity',
            'ent_city',
            'ent_state',
            'ent_postcode',
            'ent_phone_number',
            'charge_city',
            'charge_state',
            'region',
            'region_city',
            'region_state',
            'case_name',
            'case_number',
            'role']

# create a list of columns that are in the df but not in the specified new order
reordered_columns = new_order + [col for col in main_df.columns if col not in new_order]

In [5]:
# apply order
main_df = main_df[reordered_columns]

In [6]:
main_df.head()

,entity,ent_city,ent_state,ent_postcode,ent_phone_number,charge_city,charge_state,region,region_city,region_state,case_name,case_number,role
0,pacific weather,port angeles,wa,98362,NaN,port angeles,wa,3,buffalo,new york,"""pacific weather """,03-ca-027869,cdp_re
1,clp resources,tacoma,wa,98402,NaN,tacoma,wa,3,buffalo,new york,"""clp resources and controlled environmental st...",03-ca-122609,cdp_re
2,graham construction management,seattle,wa,98115,NaN,seattle,wa,3,buffalo,new york,"""graham construction & management """,03-ca-143167,cdp_re
3,ongaro burtt louderback,tacoma,wa,98402,NaN,tacoma,wa,4,philadelphia,pennsylvania,"""true blue f/k/a labor ready and its wholly ...",04-ca-075160,cdp_re
4,mcfarland cascade pole and lumber company,tacoma,wa,98421,NaN,tacoma,wa,4,philadelphia,pennsylvania,"""mcfarland cascade pole and lumber company """,04-rc-067350,e_e


In [7]:
main_df['ent_phone_number'] = main_df['ent_phone_number'].astype(str)

# clean and convert phone numbers to numeric
main_df['ent_phone_number'] = main_df['ent_phone_number'].str.replace(r'[^\d]', '', regex=True).astype(str)
main_df['ent_phone_number'] = pd.to_numeric(main_df['ent_phone_number'], errors='coerce')

In [8]:
# convert relevant columns to numeric
numeric_columns = ['ent_postcode', 'ent_phone_number']
for col in numeric_columns:
    main_df[col] = pd.to_numeric(main_df[col], errors='coerce')

In [9]:
# convert numeric columns to integers
integer_columns = ['ent_postcode','ent_phone_number']
main_df[integer_columns] = main_df[integer_columns].astype(pd.Int64Dtype())

In [10]:
# define categorical columns
categorical_columns = [
    'charge_city', 'charge_state', 'region_city', 'region_state', 'ent_city', 'ent_state','role']

# fill nan values for categorical columns with empty strings
main_df[categorical_columns] = main_df[categorical_columns].fillna('')

# convert categorical columns to string
main_df[categorical_columns] = main_df[categorical_columns].astype(str)

In [11]:
# check data types of all columns in main_df
print(main_df.dtypes)

# find all columns with mixed types or unexpected values
for col in main_df.columns:
    if main_df[col].apply(lambda x: isinstance(x, str)).any():
        print(f"Column {col} contains string values")

entity              object
ent_city            object
ent_state           object
ent_postcode         Int64
ent_phone_number     Int64
charge_city         object
charge_state        object
region               int64
region_city         object
region_state        object
case_name           object
case_number         object
role                object
dtype: object
Column entity contains string values
Column ent_city contains string values
Column ent_state contains string values
Column charge_city contains string values
Column charge_state contains string values
Column region_city contains string values
Column region_state contains string values
Column case_name contains string values
Column case_number contains string values
Column role contains string values


In [12]:
# list of columns that should be strings
string_columns = [
    'entity', 'ent_city', 'ent_state', 'charge_city', 'charge_state',
    'region_city', 'region_state', 'case_name', 'role']

# convert specified columns to string type
for col in string_columns:
    main_df[col] = main_df[col].astype(str)

# verify the conversion
print(main_df.dtypes)


entity              object
ent_city            object
ent_state           object
ent_postcode         Int64
ent_phone_number     Int64
charge_city         object
charge_state        object
region               int64
region_city         object
region_state        object
case_name           object
case_number         object
role                object
dtype: object


In [13]:
# fill or drop nan values and ensure data consistency

main_df.fillna({
    'entity': '',
    'ent_city': '',
    'ent_state': '',
    'ent_postcode': 0,
    'ent_phone_number': 0,
    'charge_city': '',
    'charge_state': '',
    'region': 0,
    'region_city': '',
    'region_state': '',
    'case_name': '',
    'case_number': '',
    'role': ''
}, inplace=True)

## Feature Engineering

In [14]:
# isolate tfidf to see if it works on text cols (this was created because initially we were getting an error making
# the tfidf fit into the text col and it would get rid of all of the 
# "ValueError: empty vocabulary; perhaps the documents only contain stop words")

vectorizer = TfidfVectorizer(stop_words=None)
try:
    vectorized_text = vectorizer.fit_transform(main_df['case_name'])
    print("vocab size after tfidf:", len(vectorizer.vocabulary_))
except ValueError:
    print("empty vocab")


vocab size after tfidf: 3915


In [15]:
# functon to find the number of components using PCA that preserve 90% of variance. this will be used as input
# in models that use PCA

def find_optimal_components(df, text_columns, numeric_columns, categorical_columns, max_features):

    vectorizer = TfidfVectorizer(max_features=max_features)
    text_features = vectorizer.fit_transform(df[text_columns].apply(lambda x: ' '.join(x.astype(str)), axis=1))

    # standardize numeric columns
    scaler = StandardScaler()
    numeric_features = scaler.fit_transform(df[numeric_columns])

    # one hot encoding categorical columns
    encoder = OneHotEncoder()
    categorical_features = encoder.fit_transform(df[categorical_columns])

    # combine all the features
    processed_features = pd.concat([pd.DataFrame(text_features.toarray()), pd.DataFrame(numeric_features), pd.DataFrame(categorical_features.toarray())], axis=1)

    # apply PCA for dimensionality redction with 90% explained variance
    pca = PCA(n_components=None)  # Set n_components to None initially
    pca.fit(processed_features)
    explained_variance = pca.explained_variance_ratio_
                                                                                                                            
    # find the number of components that explains 90% variance
    num_components_90 = np.where(np.cumsum(explained_variance) >= 0.9)[0][0] + 1
  
    return num_components_90

In [16]:
# find the number of components that preserve 90% of the variance 

numeric_columns = ['ent_postcode', 'ent_phone_number', 'region'] # standardscaler for numerical columns
categorical_columns = ['ent_city', 'ent_state', 'charge_city', 'charge_state', 'region_city', 'region_state', 'role']# one hot encoder for categorical columns
text_columns = ['case_name']  # tf-idf vectorizer for text column
max_features = 1000

pca_n_components = find_optimal_components(main_df, text_columns, numeric_columns, categorical_columns, max_features)

In [17]:
pca_n_components

164

pca_n_components = 164

In [18]:
def find_optimal_clusters(df, text_columns, numeric_columns, categorical_columns, n_clusters_range, max_features, n_components):
    """
    find optimal number of clusters using PCA

    """
    # vectorize using tfidf
    vectorizer = TfidfVectorizer(max_features=max_features)
    text_features = vectorizer.fit_transform(df[text_columns].apply(lambda x: ' '.join(x.astype(str)), axis=1))

    # standardize numeric columns
    scaler = StandardScaler()
    numeric_features = scaler.fit_transform(df[numeric_columns])

    # one hot encoding categorical columns
    encoder = OneHotEncoder()
    categorical_features = encoder.fit_transform(df[categorical_columns])

    # combine all the features
    processed_features = pd.concat([pd.DataFrame(text_features.toarray()), pd.DataFrame(numeric_features), pd.DataFrame(categorical_features.toarray())], axis=1)

    # apply PCA for dimensionality reduction
    pca = PCA(n_components=n_components)
    pca_features = pca.fit_transform(processed_features)

    # find optimal clusters using silhoutette method
    sil_scores = []
    for k in n_clusters_range:
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(pca_features)
        labels = kmeans.labels_
        sil_scores.append(silhouette_score(pca_features, labels))

    # find optimal clusters using davies-bouldin index
    db_scores = []
    for k in n_clusters_range:
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(pca_features)
        labels = kmeans.labels_
        db_scores.append(davies_bouldin_score(pca_features, labels))

    # find optimal clusters using calinski-harabasz index
    ch_scores = []
    for k in n_clusters_range:
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(pca_features)
        labels = kmeans.labels_
        ch_scores.append(calinski_harabasz_score(pca_features, labels))

    # find optimal clusters using avg within sum of square method (elbow method) (AWSS)
    awss_values = []
    for k in n_clusters_range:
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(pca_features)
        awss_values.append(kmeans.inertia_)

    # plot elbow method
    plt.figure(figsize=(10, 6))
    plt.plot(n_clusters_range, awss_values, marker='o')
    plt.xlabel('Number of Clusters')
    plt.ylabel('AWSS')
    plt.title('Elbow Method with AWSS')
    plt.xticks(n_clusters_range)
    plt.grid()
    plt.show()

    # list optimum num of clusters based on different methods
    optimal_n_clusters_silhouette = n_clusters_range[np.argmax(sil_scores)]
    optimal_n_clusters_gap = visualizer.elbow_value_
    optimal_n_clusters_davies_bouldin = n_clusters_range[np.argmin(db_scores)]
    optimal_n_clusters_calinski_harabasz = n_clusters_range[np.argmax(ch_scores)]
    optimal_n_clusters_awss = n_clusters_range[np.argmin(awss_values)]

    return {
        "silhouette method": optimal_n_clusters_silhouette,
        "gap stat": optimal_n_clusters_gap,
        "davies bouldin index": optimal_n_clusters_davies_bouldin,
        "calinski harabasz Index": optimal_n_clusters_calinski_harabasz,
        "AWSS elbow method": optimal_n_clusters_awss
    }



In [19]:
numeric_columns = ['ent_postcode', 'ent_phone_number', 'region']
categorical_columns = ['ent_city', 'ent_state', 'charge_city', 'charge_state', 'region_city', 'region_state', 'role']
text_columns = ['case_name']  
n_clusters_range = range(10, 1000)
max_features = 1000
n_components = pca_n_components 

In [20]:
# DO NOT RUN - IT TAKES TOO LONG TO FIND THE CORRECT NUMBER OF CLUSTERS

# find_optimal_clusters(main_df, text_columns, numeric_columns, categorical_columns, n_clusters_range, max_features, n_components)


Given that the aboce does not work, we can use fuzzywuzzy matching

In [21]:
entities = main_df['entity'].tolist()

# function to find number of unique entity names by clustering entities spelt similarly
def cluster_entities(entities, threshold):
    unique_entities = []
    for entity in entities:
        match = process.extractOne(entity, unique_entities, scorer=fuzz.token_sort_ratio)
        if match and match[1] >= threshold:
            continue
        unique_entities.append(entity)
    return unique_entities

# cluster entities and find unique entity names
unique_entities = cluster_entities(entities, threshold=90)

print(f"num of unique entities: {len(unique_entities)}")

num of unique entities: 3926


## pca_w_kmeans model

In [22]:
def unsupervised_pca_kmeans(df, text_columns, numeric_columns, categorical_columns, target_column, n_clusters, max_features, n_components, threshold=90):
    """
    Standardize names in the target column using PCA and KMeans clustering.
    """
    # make a copy of df not to edit main df
    df_copy = df.copy()

    # vectorize using tfidf
    vectorizer = TfidfVectorizer(max_features=max_features)
    text_features = vectorizer.fit_transform(df_copy[text_columns].apply(lambda x: ' '.join(x), axis=1))

    # standardize numeric columns
    scaler = StandardScaler()
    numeric_features = scaler.fit_transform(df_copy[numeric_columns])

    # apply one-hot encoding for categorical columns
    encoder = OneHotEncoder()
    categorical_features = encoder.fit_transform(df_copy[categorical_columns])

    # combine all features
    processed_features = pd.concat([pd.DataFrame(text_features.toarray()), pd.DataFrame(numeric_features), pd.DataFrame(categorical_features.toarray())], axis=1)

    # apply pca for dimensionality reduction
    pca = PCA(n_components=n_components)
    pca_features = pca.fit_transform(processed_features)

    # use k-means clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(pca_features)

    # assign the cluster labels to the data
    df_copy['cluster'] = clusters

    # find the most common name in each cluster and add it to the standardized_names dict
    standardized_names = {}
    for cluster_id in range(n_clusters):
        cluster_data = df_copy[df_copy['cluster'] == cluster_id][target_column]
        most_common_name = cluster_data.mode().iloc[0] if not cluster_data.mode().empty else "Unknown"
        standardized_names[cluster_id] = most_common_name

    # standardize name for each of the entities using the most common name detected
    df_copy['standardized_name'] = df_copy['cluster'].map(standardized_names)

    # evaluate clustering using diff metrics
    silhouette_avg = silhouette_score(pca_features, clusters)
    db_index = davies_bouldin_score(pca_features, clusters)
    ch_index = calinski_harabasz_score(pca_features, clusters)

    print(f"Silhouette Score: {silhouette_avg}")
    print(f"Davies-Bouldin Index: {db_index}")
    print(f"Calinski-Harabasz Index: {ch_index}")

    # evaluate similarity to see if the standardized name col is within 90% of similarity of the entity col 
    df_copy['similarity_score'] = df_copy.apply(lambda row: fuzz.token_sort_ratio(row[target_column], row['standardized_name']), axis=1)
    matches = df_copy[df_copy['similarity_score'] >= threshold]
    match_count = matches.shape[0]
    pct_of_similarity = match_count / len(df_copy)

    print(f"Percentage of Similarity: {pct_of_similarity}")

    return df_copy, silhouette_avg, db_index, ch_index, pct_of_similarity


In [23]:
numeric_columns = ['ent_postcode', 'ent_phone_number', 'region']
categorical_columns = ['ent_city', 'ent_state', 'charge_city', 'charge_state', 'region_city', 'region_state', 'role']
text_columns = ['case_name']  
target_column = 'entity'
n_clusters = len(unique_entities)
max_features = 1000
n_components = pca_n_components 


pca_kmeans_df, pca_kmeans_silhouette, pca_kmeans_db_index, pca_kmeans_ch_index, pca_kmeans_pca_similarity = unsupervised_pca_kmeans(main_df, text_columns, numeric_columns, categorical_columns, target_column, n_clusters, max_features, n_components, threshold=90)

Silhouette Score: 0.5723492859588513
Davies-Bouldin Index: 0.5982182251751493
Calinski-Harabasz Index: 486.97404638068906
Percentage of Similarity: 0.5647598739106249


In [24]:
pca_kmeans_df

,entity,ent_city,ent_state,ent_postcode,ent_phone_number,charge_city,charge_state,region,region_city,region_state,case_name,case_number,role,cluster,standardized_name,similarity_score
0,pacific weather,port angeles,wa,98362,0,port angeles,wa,3,buffalo,new york,"""pacific weather """,03-ca-027869,cdp_re,1527,pacific weather,100.000000
1,clp resources,tacoma,wa,98402,0,tacoma,wa,3,buffalo,new york,"""clp resources and controlled environmental st...",03-ca-122609,cdp_re,1253,clp resources,100.000000
2,graham construction management,seattle,wa,98115,0,seattle,wa,3,buffalo,new york,"""graham construction & management """,03-ca-143167,cdp_re,5,graham construction management,100.000000
3,ongaro burtt louderback,tacoma,wa,98402,0,tacoma,wa,4,philadelphia,pennsylvania,"""true blue f/k/a labor ready and its wholly ...",04-ca-075160,cdp_re,1656,ongaro burtt louderback,100.000000
4,mcfarland cascade pole and lumber company,tacoma,wa,98421,0,tacoma,wa,4,philadelphia,pennsylvania,"""mcfarland cascade pole and lumber company """,04-rc-067350,e_e,272,mcfarland cascade holdings a wholly owned sub...,51.239669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21567,oregon school employees association local 6732,milwaukie,or,97267,0,lake oswego,or,19,seattle,washington,"""first student """,36-ud-000376,ip_u,1493,oregon school employees association aft local,92.307692
21568,first student,lake oswego,or,97035,5035342332,lake oswego,or,19,seattle,washington,"""first student """,36-ud-000376,e_e,653,first student,100.000000
21569,first student,lake oswego,or,97035,5035342332,lake oswego,or,19,seattle,washington,"""first student """,36-ud-000376,e_e,653,first student,100.000000
21570,teamsters local 58,vancouver,wa,98661,3606935841,battle ground,wa,19,seattle,washington,"""petermann northwest """,36-ud-000377,ip_u,1095,teamsters local 58,100.000000


## pca_w_agglomerative model

In [25]:
def unsupervised_pca_agglomerative(df, text_columns, numeric_columns, categorical_columns, target_column, max_features, n_components, distance_threshold, threshold=90):
    """
    Standardize names in the target column using PCA and Agglomerative clustering.
    """
    # make a copy of df not to lose
    df_copy = df.copy()

    # vectorize text columns using tfidf
    vectorizer = TfidfVectorizer(max_features=max_features)
    text_features = vectorizer.fit_transform(df_copy[text_columns].apply(lambda x: ' '.join(x), axis=1))

    # standardize numeric columns
    scaler = StandardScaler()
    numeric_features = scaler.fit_transform(df_copy[numeric_columns])

    # apply one hot encoding for categorical columns
    encoder = OneHotEncoder()
    categorical_features = encoder.fit_transform(df_copy[categorical_columns])

    # combine all features
    processed_features = np.hstack([text_features.toarray(), numeric_features, categorical_features.toarray()])

    # apply pca for dimensionality reduction
    pca = PCA(n_components=n_components)
    pca_features = pca.fit_transform(processed_features)

    # use agglomerative clustering with distance_threshold
    agglomerative = AgglomerativeClustering(distance_threshold=distance_threshold, n_clusters=None)
    clusters = agglomerative.fit_predict(pca_features)

    # assign the cluster labels to the data
    df_copy['cluster'] = clusters

    # find the most common name in each cluster and add it to the standardized_names dict
    standardized_names = {}
    unique_clusters = df_copy['cluster'].unique()
    for cluster_id in unique_clusters:
        cluster_data = df_copy[df_copy['cluster'] == cluster_id][target_column]
        most_common_name = cluster_data.mode().iloc[0] if not cluster_data.mode().empty else "Unknown"
        standardized_names[cluster_id] = most_common_name

    # standardize name for each of the entities using the most common name detected
    df_copy['standardized_name'] = df_copy['cluster'].map(standardized_names)

    # evaluate using different metrics
    silhouette_avg = silhouette_score(pca_features, clusters)
    db_index = davies_bouldin_score(pca_features, clusters)
    ch_index = calinski_harabasz_score(pca_features, clusters)

    print(f"Silhouette Score: {silhouette_avg}")
    print(f"Davies-Bouldin Index: {db_index}")
    print(f"Calinski-Harabasz Index: {ch_index}")

    # evaluate similarity to see if the standardized name col is within 90% of similarity of the entity col 
    df_copy['similarity_score'] = df_copy.apply(lambda row: fuzz.token_sort_ratio(row[target_column], row['standardized_name']), axis=1)
    matches = df_copy[df_copy['similarity_score'] >= threshold]
    match_count = matches.shape[0]
    pct_of_similarity = match_count / len(df_copy)

    print(f"Percentage of similarity: {pct_of_similarity}")

    return df_copy, silhouette_avg, db_index, ch_index, pct_of_similarity


In [26]:
numeric_columns = ['ent_postcode', 'ent_phone_number', 'region']
categorical_columns = ['ent_city', 'ent_state', 'charge_city', 'charge_state', 'region_city', 'region_state', 'role']
text_columns = ['case_name']  
target_column = 'entity'
max_features = 1000
n_components = pca_n_components 
distance_threshold = 0.1


pca_agg_df, pca_agg_silhouette, pca_agg_db_index, pca_agg_ch_index, pca_agg_similarity = unsupervised_pca_agglomerative(main_df, text_columns, numeric_columns, categorical_columns, target_column, max_features, n_components, distance_threshold=distance_threshold, threshold=90)


Silhouette Score: 0.6453628185406257
Davies-Bouldin Index: 0.06677569657450089
Calinski-Harabasz Index: 99935.2216992327
Percentage of similarity: 0.8349249026515854


In [27]:
pca_agg_df

,entity,ent_city,ent_state,ent_postcode,ent_phone_number,charge_city,charge_state,region,region_city,region_state,case_name,case_number,role,cluster,standardized_name,similarity_score
0,pacific weather,port angeles,wa,98362,0,port angeles,wa,3,buffalo,new york,"""pacific weather """,03-ca-027869,cdp_re,6911,pacific weather,100.0
1,clp resources,tacoma,wa,98402,0,tacoma,wa,3,buffalo,new york,"""clp resources and controlled environmental st...",03-ca-122609,cdp_re,5751,clp resources,100.0
2,graham construction management,seattle,wa,98115,0,seattle,wa,3,buffalo,new york,"""graham construction & management """,03-ca-143167,cdp_re,6627,graham construction management,100.0
3,ongaro burtt louderback,tacoma,wa,98402,0,tacoma,wa,4,philadelphia,pennsylvania,"""true blue f/k/a labor ready and its wholly ...",04-ca-075160,cdp_re,9523,ongaro burtt louderback,100.0
4,mcfarland cascade pole and lumber company,tacoma,wa,98421,0,tacoma,wa,4,philadelphia,pennsylvania,"""mcfarland cascade pole and lumber company """,04-rc-067350,e_e,6451,mcfarland cascade pole and lumber company,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21567,oregon school employees association local 6732,milwaukie,or,97267,0,lake oswego,or,19,seattle,washington,"""first student """,36-ud-000376,ip_u,1347,oregon school employees association local 6732,100.0
21568,first student,lake oswego,or,97035,5035342332,lake oswego,or,19,seattle,washington,"""first student """,36-ud-000376,e_e,2276,first student,100.0
21569,first student,lake oswego,or,97035,5035342332,lake oswego,or,19,seattle,washington,"""first student """,36-ud-000376,e_e,2276,first student,100.0
21570,teamsters local 58,vancouver,wa,98661,3606935841,battle ground,wa,19,seattle,washington,"""petermann northwest """,36-ud-000377,ip_u,4809,teamsters local 58,100.0


## tsne_w_kmeans model

In [28]:
def unsupervised_tsne_kmeans(df, text_columns, numeric_columns, categorical_columns, target_column, n_clusters, max_features, n_components, threshold=90):
    """
    Standardize names in the target column using t-SNE and K-Means clustering.
    """
    
    df_copy = df.copy()

    # vectorize using tfidf 
    vectorizer = TfidfVectorizer(max_features=max_features)
    text_features = vectorizer.fit_transform(df_copy[text_columns].apply(lambda x: ' '.join(x), axis=1))

    # standardize numeric columns
    scaler = StandardScaler()
    numeric_features = scaler.fit_transform(df_copy[numeric_columns])

    # apply one-hot encoding for categorical columns
    encoder = OneHotEncoder()
    categorical_features = encoder.fit_transform(df_copy[categorical_columns])

    # combine all features
    processed_features = hstack([text_features, numeric_features, categorical_features])

    # apply t-SNE for dimension reduction
    tsne = TSNE(n_components=n_components, random_state=42)
    tsne_features = tsne.fit_transform(processed_features.toarray())

    # use kmeans clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(tsne_features)

    # assign the cluster labels to the data
    df_copy['cluster'] = clusters

    # find the most common name in each cluster and add it to the standardized_names dict
    standardized_names = {}
    for cluster_id in range(n_clusters):
        cluster_data = df_copy[df_copy['cluster'] == cluster_id][target_column]
        most_common_name = cluster_data.mode().iloc[0] if not cluster_data.mode().empty else "Unknown"
        standardized_names[cluster_id] = most_common_name

    # standardize name for each of the entities using the most common name detected
    df_copy['standardized_name'] = df_copy['cluster'].map(standardized_names)

    # eval using diff eval metrics
    silhouette_avg = silhouette_score(tsne_features, clusters)
    db_index = davies_bouldin_score(tsne_features, clusters)
    ch_index = calinski_harabasz_score(tsne_features, clusters)

    print(f"Silhouette Score: {silhouette_avg}")
    print(f"Davies-Bouldin Index: {db_index}")
    print(f"Calinski-Harabasz Index: {ch_index}")

    # evaluate similarity to see if the standardized name col is within 90% of similarity of the entity col 
    df_copy['similarity_score'] = df_copy.apply(lambda row: fuzz.token_sort_ratio(row[target_column], row['standardized_name']), axis=1)
    matches = df_copy[df_copy['similarity_score'] >= threshold]
    match_count = matches.shape[0]
    pct_of_similarity = match_count / len(df_copy)

    print(f"Percentage of similarity: {pct_of_similarity}")

    return df_copy, silhouette_avg, db_index, ch_index, pct_of_similarity


In [29]:
numeric_columns = ['ent_postcode', 'ent_phone_number', 'region']
categorical_columns = ['ent_city', 'ent_state', 'charge_city', 'charge_state', 'region_city', 'region_state', 'role']
text_columns = ['case_name']  
target_column = 'entity'
n_clusters = len(unique_entities)
max_features = 1000
n_components = 2

tsne_kmeans_df, tsne_kmeans_silhouette, tsne_kmeans_db_index, tsne_kmeans_ch_index, tsne_kmeans_pct_of_similarity = unsupervised_tsne_kmeans(main_df, text_columns, numeric_columns, categorical_columns, target_column, n_clusters, max_features, n_components, threshold=90)

Silhouette Score: 0.7642533183097839
Davies-Bouldin Index: 0.30283005256912426
Calinski-Harabasz Index: 316315.6731906529
Percentage of similarity: 0.6160763953272761


In [30]:
tsne_kmeans_df

,entity,ent_city,ent_state,ent_postcode,ent_phone_number,charge_city,charge_state,region,region_city,region_state,case_name,case_number,role,cluster,standardized_name,similarity_score
0,pacific weather,port angeles,wa,98362,0,port angeles,wa,3,buffalo,new york,"""pacific weather """,03-ca-027869,cdp_re,193,ziply fiber,30.769231
1,clp resources,tacoma,wa,98402,0,tacoma,wa,3,buffalo,new york,"""clp resources and controlled environmental st...",03-ca-122609,cdp_re,193,ziply fiber,16.666667
2,graham construction management,seattle,wa,98115,0,seattle,wa,3,buffalo,new york,"""graham construction & management """,03-ca-143167,cdp_re,193,ziply fiber,14.634146
3,ongaro burtt louderback,tacoma,wa,98402,0,tacoma,wa,4,philadelphia,pennsylvania,"""true blue f/k/a labor ready and its wholly ...",04-ca-075160,cdp_re,193,ziply fiber,23.529412
4,mcfarland cascade pole and lumber company,tacoma,wa,98421,0,tacoma,wa,4,philadelphia,pennsylvania,"""mcfarland cascade pole and lumber company """,04-rc-067350,e_e,193,ziply fiber,23.076923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21567,oregon school employees association local 6732,milwaukie,or,97267,0,lake oswego,or,19,seattle,washington,"""first student """,36-ud-000376,ip_u,3045,oregon school employees association aft local,92.307692
21568,first student,lake oswego,or,97035,5035342332,lake oswego,or,19,seattle,washington,"""first student """,36-ud-000376,e_e,1095,first student,100.000000
21569,first student,lake oswego,or,97035,5035342332,lake oswego,or,19,seattle,washington,"""first student """,36-ud-000376,e_e,1095,first student,100.000000
21570,teamsters local 58,vancouver,wa,98661,3606935841,battle ground,wa,19,seattle,washington,"""petermann northwest """,36-ud-000377,ip_u,1955,teamsters local 58,100.000000


## tsne_w_agglomerative model

In [31]:
def unsupervised_tsne_agglomerative(df, text_columns, numeric_columns, categorical_columns, target_column, max_features, n_components, distance_threshold, threshold=90):
    """
    Standardize names in the target column using t-SNE and agglomerative clustering.
    """
    
    # make a copy of df
    df_copy = df.copy()

    # vectorize text columns using tfidf
    vectorizer = TfidfVectorizer(max_features=max_features)
    text_features = vectorizer.fit_transform(df_copy[text_columns].apply(lambda x: ' '.join(x), axis=1))

    # standardize numeric columns
    scaler = StandardScaler()
    numeric_features = scaler.fit_transform(df_copy[numeric_columns])

    # apply one hot encoding for categorical columns
    encoder = OneHotEncoder()
    categorical_features = encoder.fit_transform(df_copy[categorical_columns])

    # combine all features
    processed_features = hstack([text_features, numeric_features, categorical_features])

    # apply tsne for dimensionality reduction
    tsne = TSNE(n_components=n_components, random_state=42)
    tsne_features = tsne.fit_transform(processed_features.toarray())

    # use agglomerative clustering
    agglomerative = AgglomerativeClustering(distance_threshold=distance_threshold, n_clusters=None)
    clusters = agglomerative.fit_predict(tsne_features)

    # assign the cluster labels to the data
    df_copy['cluster'] = clusters

    # find the most common name in each cluster and add it to the standardized_names dict
    standardized_names = {}
    unique_clusters = df_copy['cluster'].unique()
    for cluster_id in unique_clusters:
        cluster_data = df_copy[df_copy['cluster'] == cluster_id][target_column]
        most_common_name = cluster_data.mode().iloc[0] if not cluster_data.mode().empty else "Unknown"
        standardized_names[cluster_id] = most_common_name

    # standardize name for each of the entities using the most common name detected
    df_copy['standardized_name'] = df_copy['cluster'].map(standardized_names)

    # evaluate using diff metrics
    silhouette_avg = silhouette_score(tsne_features, clusters)
    db_index = davies_bouldin_score(tsne_features, clusters)
    ch_index = calinski_harabasz_score(tsne_features, clusters)

    print(f"Silhouette Score: {silhouette_avg}")
    print(f"Davies-Bouldin Index: {db_index}")
    print(f"Calinski-Harabasz Index: {ch_index}")

    # evaluate similarity to see if the standardized name col is within 90% of similarity of the entity col 
    df_copy['similarity_score'] = df_copy.apply(lambda row: fuzz.token_sort_ratio(row[target_column], row['standardized_name']), axis=1)
    matches = df_copy[df_copy['similarity_score'] >= threshold]
    match_count = matches.shape[0]
    pct_of_similarity = match_count / len(df_copy)

    print(f"Percentage of similarity: {pct_of_similarity}")

    return df_copy, silhouette_avg, db_index, ch_index, pct_of_similarity


In [32]:
numeric_columns = ['ent_postcode', 'ent_phone_number', 'region']
categorical_columns = ['ent_city', 'ent_state', 'charge_city', 'charge_state', 'region_city', 'region_state', 'role']
text_columns = ['case_name']  
target_column = 'entity'
max_features = 1000
n_components = 2
distance_threshold = 0.1

tsne_agg_df, tsne_agg_silhouette, tsne_agg_db_index, tsne_agg_ch_index, tsne_agg_pct_similarity = unsupervised_tsne_agglomerative(main_df, text_columns, numeric_columns, categorical_columns, target_column, max_features, n_components, distance_threshold=distance_threshold, threshold=90)


Silhouette Score: 0.7536448836326599
Davies-Bouldin Index: 0.06417127259147974
Calinski-Harabasz Index: 50378435.61918042
Percentage of similarity: 0.7983960689783052


## Sensitivity Analysis

### sensitivity analysis for pca_w_kmeans model

In [33]:
# specify num of components range, for our main models we use 164
n_components_range = [100, 164, 200]

# specify num of components range, for our main models we use 164
max_features_range = [100, 1000, 10000]

pca_kmeans_results = []

# loop across specified parameters
for n_components in n_components_range:
    for max_features in max_features_range:
        print(f"Running for n_components={n_components}, max_features={max_features}")
        try:
            # run the unsupervised_pca_kmeans function
            pca_kmeans_df, pca_kmeans_silhouette, pca_kmeans_db_index, pca_kmeans_ch_index, pca_kmeans_pca_similarity = unsupervised_pca_kmeans(
                main_df,text_columns, numeric_columns, categorical_columns, target_column, n_clusters=3926, max_features=max_features, n_components=n_components, threshold=90)
            
            # store results
            pca_kmeans_results.append({
                'n_components': n_components,
                'max_features': max_features,
                'silhouette_avg': pca_kmeans_silhouette,
                'db_index': pca_kmeans_db_index,
                'ch_index': pca_kmeans_ch_index,
                'pct_of_similarity': pca_kmeans_pca_similarity})
        except Exception as e:
            print(f"Failed for n_components={n_components}, max_features={max_features}: {e}")

pca_kmeans_results_df = pd.DataFrame(pca_kmeans_results)

print(pca_kmeans_results_df)


Running for n_components=100, max_features=100
Silhouette Score: 0.6601953122654876
Davies-Bouldin Index: 0.6050801783392146
Calinski-Harabasz Index: 447.3146334139011
Percentage of Similarity: 0.5581309104394585
Running for n_components=100, max_features=1000
Silhouette Score: 0.5935605538670321
Davies-Bouldin Index: 0.4787256846475272
Calinski-Harabasz Index: 1176.7296692520006
Percentage of Similarity: 0.5485814945299462
Running for n_components=100, max_features=10000
Silhouette Score: 0.6073191077197236
Davies-Bouldin Index: 0.38712617441215924
Calinski-Harabasz Index: 2247.577788863425
Percentage of Similarity: 0.5470053773409976
Running for n_components=164, max_features=100
Silhouette Score: 0.6522835727565083
Davies-Bouldin Index: 0.6987583296387304
Calinski-Harabasz Index: 261.3702109080762
Percentage of Similarity: 0.5696272946412015
Running for n_components=164, max_features=1000
Silhouette Score: 0.5666709781294453
Davies-Bouldin Index: 0.6035252665342752
Calinski-Harabasz

### sensitivity analysis for pca_w_agglomerative model

In [34]:
# specify num of components range, for our main models we use 164
n_components_range = [100, 164, 200]

# specify num of components range, for our main models we use 164
max_features_range = [100, 1000, 10000]

pca_agglomerative_results = []

# loop across specified parameters
for n_components in n_components_range:
    for max_features in max_features_range:
        print(f"Running for n_components={n_components}, max_features={max_features}")
        try:
            # run the unsupervised_pca_agglomerative function
            pca_agg_df, pca_agg_silhouette, pca_agg_db_index, pca_agg_ch_index, pca_agg_similarity = unsupervised_pca_agglomerative(main_df, text_columns, 
            numeric_columns, categorical_columns, target_column, max_features, n_components, distance_threshold=0.1, threshold=90)

            
            # store the results
            pca_agglomerative_results.append({
                'n_components': n_components,
                'max_features': max_features,
                'silhouette_avg': pca_agg_silhouette,
                'db_index': pca_agg_db_index,
                'ch_index': pca_agg_ch_index,
                'pct_of_similarity': pca_agg_similarity})
        except Exception as e:
            print(f"Failed for n_components={n_components}, max_features={max_features}: {e}")

pca_agglomerative_results_df = pd.DataFrame(pca_agglomerative_results)

print(pca_agglomerative_results_df)

Running for n_components=100, max_features=100
Silhouette Score: 0.7330897011247874
Davies-Bouldin Index: 0.04433899171519896
Calinski-Harabasz Index: 256816.54931957743
Percentage of similarity: 0.7356295197478212
Running for n_components=100, max_features=1000
Silhouette Score: 0.6486352903084565
Davies-Bouldin Index: 0.10858176608089357
Calinski-Harabasz Index: 61145.452011616544
Percentage of similarity: 0.7984424253662155
Running for n_components=100, max_features=10000
Silhouette Score: 0.6449110737289058
Davies-Bouldin Index: 0.13509365050895164
Calinski-Harabasz Index: 53602.219050636864
Percentage of similarity: 0.7745688855924346
Running for n_components=164, max_features=100
Silhouette Score: 0.7312064486652984
Davies-Bouldin Index: 0.0320956870458932
Calinski-Harabasz Index: 397983.68367622345
Percentage of similarity: 0.7444835898386798
Running for n_components=164, max_features=1000
Silhouette Score: 0.6453621189734835
Davies-Bouldin Index: 0.06596890418723662
Calinski-Ha

### sensitivity analysis for tsne_w_kmeans model

In [35]:
# specify num of components range, for our main models we use 206
n_components_range = [1, 2]

# specify num of components range, for our main models we use 206
max_features_range = [100, 1000, 10000]

tsne_kmeans_results = []

# loop across specified parameters
for n_components in n_components_range:
    for max_features in max_features_range:
        print(f"Running for n_components={n_components}, max_features={max_features}")
        try:
            # run the unsupervised_tsne_kmeans function
            tsne_kmeans_df, tsne_kmeans_silhouette, tsne_kmeans_db_index, tsne_kmeans_ch_index, tsne_kmeans_pca_similarity = unsupervised_tsne_kmeans(
                main_df, text_columns, numeric_columns, categorical_columns, target_column, 3926, max_features, n_components)
            
            # store the results
            tsne_kmeans_results.append({
                'n_components': n_components,
                'max_features': max_features,
                'silhouette_avg': tsne_kmeans_silhouette,
                'db_index': tsne_kmeans_db_index,
                'ch_index': tsne_kmeans_ch_index,
                'pct_of_similarity': tsne_kmeans_pca_similarity})
        except Exception as e:
            print(f"Failed for n_components={n_components}, max_features={max_features}: {e}")

tsne_kmeans_results_df = pd.DataFrame(tsne_kmeans_results)

print(tsne_kmeans_results_df)

Running for n_components=1, max_features=100


/home/snazaral/.local/lib/python3.9/site-packages/sklearn/base.py:1473: ConvergenceWarning: Number of distinct clusters (3838) found smaller than n_clusters (3926). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


Silhouette Score: 0.7243363261222839
Davies-Bouldin Index: 0.22749304397561393
Calinski-Harabasz Index: 811837696.3350518
Percentage of similarity: 0.5322640459855368
Running for n_components=1, max_features=1000


/home/snazaral/.local/lib/python3.9/site-packages/sklearn/base.py:1473: ConvergenceWarning: Number of distinct clusters (3811) found smaller than n_clusters (3926). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


Silhouette Score: 0.6913195252418518
Davies-Bouldin Index: 0.27575929598796645
Calinski-Harabasz Index: 620715452.2100738
Percentage of similarity: 0.5504821064342666
Running for n_components=1, max_features=10000


/home/snazaral/.local/lib/python3.9/site-packages/sklearn/base.py:1473: ConvergenceWarning: Number of distinct clusters (3888) found smaller than n_clusters (3926). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


Silhouette Score: 0.6683393716812134
Davies-Bouldin Index: 0.2779117399270478
Calinski-Harabasz Index: 417435071.25588703
Percentage of similarity: 0.5439458557389208
Running for n_components=2, max_features=100
Silhouette Score: 0.7784396409988403
Davies-Bouldin Index: 0.2741859403756169
Calinski-Harabasz Index: 444908.5880011939
Percentage of similarity: 0.5853884665306879
Running for n_components=2, max_features=1000
Silhouette Score: 0.7642533183097839
Davies-Bouldin Index: 0.30283005256912426
Calinski-Harabasz Index: 316315.6731906529
Percentage of similarity: 0.6160763953272761
Running for n_components=2, max_features=10000
Silhouette Score: 0.7510783076286316
Davies-Bouldin Index: 0.3155468187841202
Calinski-Harabasz Index: 241054.3597761237
Percentage of similarity: 0.617698868904135
   n_components  max_features  silhouette_avg  db_index      ch_index  \
0             1           100        0.724336  0.227493  8.118377e+08   
1             1          1000        0.691320  0.27

### sensitivity analysis for tsne_w_agglomerative model

In [36]:
# specify num of components range, for our main models we use 206
n_components_range = [1, 2]

# specify num of components range, for our main models we use 206
max_features_range = [100, 1000, 10000]

tsne_agglomerative_results = []

# loop across specified parameters
for n_components in n_components_range:
    for max_features in max_features_range:
        print(f"Running for n_components={n_components}, max_features={max_features}")
        try:
            # run the unsupervised_tsne_agglomerative function
            tsne_agglomerative_df, tsne_agglomerative_silhouette, tsne_agglomerative_db_index, tsne_agglomerative_ch_index, tsne_agglomerative_similarity = unsupervised_tsne_agglomerative(main_df, text_columns, numeric_columns, categorical_columns, target_column, max_features, n_components, distance_threshold=0.1, threshold=90)

            
            # store the results
            tsne_agglomerative_results.append({
                'n_components': n_components,
                'max_features': max_features,
                'silhouette_avg': tsne_agglomerative_silhouette,
                'db_index': tsne_agglomerative_db_index,
                'ch_index': tsne_agglomerative_ch_index,
                'pct_of_similarity': tsne_agglomerative_similarity})
        except Exception as e:
            print(f"Failed for n_components={n_components}, max_features={max_features}: {e}")

tsne_agglomerative_results_df = pd.DataFrame(tsne_agglomerative_results)

print(tsne_agglomerative_results_df)


Running for n_components=1, max_features=100
Silhouette Score: 0.7656919360160828
Davies-Bouldin Index: 0.2916750227711943
Calinski-Harabasz Index: 265067170.3892358
Percentage of similarity: 0.4472464305581309
Running for n_components=1, max_features=1000
Silhouette Score: 0.7427453994750977
Davies-Bouldin Index: 0.3056101142225912
Calinski-Harabasz Index: 242352385.43042535
Percentage of similarity: 0.4660207676617838
Running for n_components=1, max_features=10000
Silhouette Score: 0.7195737361907959
Davies-Bouldin Index: 0.3225651449088818
Calinski-Harabasz Index: 190851871.8636864
Percentage of similarity: 0.47019284257370664
Running for n_components=2, max_features=100
Silhouette Score: 0.7594237327575684
Davies-Bouldin Index: 0.06268721051946177
Calinski-Harabasz Index: 59135480.42114508
Percentage of similarity: 0.7490265158538847
Running for n_components=2, max_features=1000
Silhouette Score: 0.7536448836326599
Davies-Bouldin Index: 0.06417127259147974
Calinski-Harabasz Index: 

In [37]:
pca_kmeans_results_df.to_csv("pca_kmeans_results_df.csv", index=False)
pca_agglomerative_results_df.to_csv("pca_agglomerative_results_df.csv", index=False)
tsne_kmeans_results_df.to_csv("tsne_kmeans_results_df.csv", index=False)
tsne_agglomerative_results_df.to_csv("tsne_agglomerative_results_df.csv", index = False)

## Cross Validation

### cross validation for pca_w_kmeans model

In [38]:
pca_kmeans_cv_df = pd.DataFrame(columns=['Silhouette Score', 'Davies-Bouldin Index', 'Calinski-Harabasz Index', 'Percentage of Similarity'])

# run the function 5 times for cross-validation
for i in range(5):
    print(f"Running cross-validation iteration {i+1}")
    
    # call the unsupervised_pca_kmeans function
    pca_kmeans_df, pca_kmeans_silhouette, pca_kmeans_db_index, pca_kmeans_ch_index, pca_kmeans_similarity = unsupervised_pca_kmeans(
                main_df,text_columns, numeric_columns, categorical_columns, target_column, n_clusters=3926, max_features=1000, n_components=164, threshold=90)
    
    # append the scores to the df
    pca_kmeans_cv_df = pca_kmeans_cv_df.append({
        'Silhouette Score': pca_kmeans_silhouette,
        'Davies-Bouldin Index': pca_kmeans_db_index,
        'Calinski-Harabasz Index': pca_kmeans_ch_index,
        'Percentage of Similarity': pca_kmeans_similarity}, ignore_index=True)

print(pca_kmeans_cv_df)

Running cross-validation iteration 1
Silhouette Score: 0.569629629719997
Davies-Bouldin Index: 0.5974334729428996
Calinski-Harabasz Index: 483.8158914443596
Percentage of Similarity: 0.5661505655479325
Running cross-validation iteration 2
Silhouette Score: 0.5694038892040079
Davies-Bouldin Index: 0.5972132892257324
Calinski-Harabasz Index: 488.2107308687488
Percentage of Similarity: 0.5669849805303171
Running cross-validation iteration 3
Silhouette Score: 0.5597895376283445
Davies-Bouldin Index: 0.60962512845157
Calinski-Harabasz Index: 481.6601286376001
Percentage of Similarity: 0.5680048210643427
Running cross-validation iteration 4
Silhouette Score: 0.5728624127575171
Davies-Bouldin Index: 0.6064417218815631
Calinski-Harabasz Index: 484.7310712519797
Percentage of Similarity: 0.5671240496940478
Running cross-validation iteration 5
Silhouette Score: 0.5705360491082948
Davies-Bouldin Index: 0.603421264300795
Calinski-Harabasz Index: 481.47099724015885
Percentage of Similarity: 0.56619

### cross validation for pca_w_agglomerative model

In [39]:
pca_agglomerative_cv_df = pd.DataFrame(columns=['Silhouette Score', 'Davies-Bouldin Index', 'Calinski-Harabasz Index', 'Percentage of Similarity'])

# run the function 5 times for cross-validation
for i in range(5):
    print(f"Running cross-validation iteration {i+1}")
    
    try:
        # call the unsupervised_pca_agglomerative function
        pca_agglomerative_df, pca_agglomerative_silhouette, pca_agglomerative_db_index, pca_agglomerative_ch_index, pca_agglomerative_similarity = unsupervised_pca_agglomerative(main_df, text_columns, numeric_columns, categorical_columns, target_column, max_features=1000, n_components = 164, distance_threshold=0.1, threshold=90)
        
        # append the scores to the df
        pca_agglomerative_cv_df = pca_agglomerative_cv_df.append({
            'Silhouette Score': pca_agglomerative_silhouette,
            'Davies-Bouldin Index': pca_agglomerative_db_index,
            'Calinski-Harabasz Index': pca_agglomerative_ch_index,
            'Percentage of Similarity': pca_agglomerative_similarity}, ignore_index=True)
    
    except Exception as e:
        print(f"Failed for cross-validation iteration {i+1}: {e}")

print(pca_agglomerative_cv_df)

Running cross-validation iteration 1
Silhouette Score: 0.6453638166598811
Davies-Bouldin Index: 0.06756138430155577
Calinski-Harabasz Index: 101470.58672569836
Percentage of similarity: 0.8353421101427777
Running cross-validation iteration 2
Silhouette Score: 0.6447444349821437
Davies-Bouldin Index: 0.06721271544571986
Calinski-Harabasz Index: 98803.83466839336
Percentage of similarity: 0.8344149823845726
Running cross-validation iteration 3
Silhouette Score: 0.6460453960986113
Davies-Bouldin Index: 0.06501747157814121
Calinski-Harabasz Index: 102996.19918704794
Percentage of similarity: 0.8357593176339699
Running cross-validation iteration 4
Silhouette Score: 0.6449449000317361
Davies-Bouldin Index: 0.06714219145791682
Calinski-Harabasz Index: 101391.19703897175
Percentage of similarity: 0.8354348229185982
Running cross-validation iteration 5
Silhouette Score: 0.6454665451749377
Davies-Bouldin Index: 0.06673031433850919
Calinski-Harabasz Index: 100029.24868106093
Percentage of similar

### cross validation for tsne_w_kmeans model

In [40]:
tsne_kmeans_cv_df = pd.DataFrame(columns=['Silhouette Score', 'Davies-Bouldin Index', 'Calinski-Harabasz Index', 'Percentage of Similarity'])

# run the function 5 times for cross-validation
for i in range(5):
    print(f"Running cross-validation iteration {i+1}")
    
    try:
        # call the unsupervised_tsne_kmeans function
        tsne_kmeans_df, tsne_kmeans_silhouette, tsne_kmeans_db_index, tsne_kmeans_ch_index, tsne_kmeans_similarity = unsupervised_tsne_kmeans(
            main_df, text_columns, numeric_columns, categorical_columns, target_column, n_clusters=3926, max_features=1000, n_components=2)
        
        # append the scores to the df
        tsne_kmeans_cv_df = tsne_kmeans_cv_df.append({
            'Silhouette Score': tsne_kmeans_silhouette,
            'Davies-Bouldin Index': tsne_kmeans_db_index,
            'Calinski-Harabasz Index': tsne_kmeans_ch_index,
            'Percentage of Similarity': tsne_kmeans_similarity}, ignore_index=True)
    
    except Exception as e:
        print(f"Failed for cross-validation iteration {i+1}: {e}")

print(tsne_kmeans_cv_df)

Running cross-validation iteration 1
Silhouette Score: 0.7642533183097839
Davies-Bouldin Index: 0.30283005256912426
Calinski-Harabasz Index: 316315.6731906529
Percentage of similarity: 0.6160763953272761
Running cross-validation iteration 2
Silhouette Score: 0.7642533183097839
Davies-Bouldin Index: 0.30283005256912426
Calinski-Harabasz Index: 316315.6731906529
Percentage of similarity: 0.6160763953272761
Running cross-validation iteration 3
Silhouette Score: 0.7642533183097839
Davies-Bouldin Index: 0.30283005256912426
Calinski-Harabasz Index: 316315.6731906529
Percentage of similarity: 0.6160763953272761
Running cross-validation iteration 4
Silhouette Score: 0.7642533183097839
Davies-Bouldin Index: 0.30283005256912426
Calinski-Harabasz Index: 316315.6731906529
Percentage of similarity: 0.6160763953272761
Running cross-validation iteration 5
Silhouette Score: 0.7642533183097839
Davies-Bouldin Index: 0.30283005256912426
Calinski-Harabasz Index: 316315.6731906529
Percentage of similarity:

### cross validation for tsne_w_agglomerative model

In [41]:
tsne_agglomerative_cv_df = pd.DataFrame(columns=['Silhouette Score', 'Davies-Bouldin Index', 'Calinski-Harabasz Index', 'Percentage of Similarity'])

# run the function 5 times for cross-validation
for i in range(5):
    print(f"Running cross-validation iteration {i+1}")
    
    try:
        # call the unsupervised_tsne_agglomerative function
        tsne_agglomerative_df, tsne_agglomerative_silhouette, tsne_agglomerative_db_index, tsne_agglomerative_ch_index, tsne_agglomerative_similarity = unsupervised_tsne_agglomerative(main_df, text_columns, numeric_columns, categorical_columns, target_column, max_features=1000, n_components=2, distance_threshold=0.1, threshold=90)
        
        # append the scores to the df
        tsne_agglomerative_cv_df = tsne_agglomerative_cv_df.append({
            'Silhouette Score': tsne_agglomerative_silhouette,
            'Davies-Bouldin Index': tsne_agglomerative_db_index,
            'Calinski-Harabasz Index': tsne_agglomerative_ch_index,
            'Percentage of Similarity': tsne_agglomerative_similarity}, ignore_index=True)
    
    except Exception as e:
        print(f"Failed for cross-validation iteration {i+1}: {e}")

print(tsne_agglomerative_cv_df)

Running cross-validation iteration 1
Silhouette Score: 0.7536448836326599
Davies-Bouldin Index: 0.06417127259147974
Calinski-Harabasz Index: 50378435.61918042
Percentage of similarity: 0.7983960689783052
Running cross-validation iteration 2
Silhouette Score: 0.7536448836326599
Davies-Bouldin Index: 0.06417127259147974
Calinski-Harabasz Index: 50378435.61918042
Percentage of similarity: 0.7983960689783052
Running cross-validation iteration 3
Silhouette Score: 0.7536448836326599
Davies-Bouldin Index: 0.06417127259147974
Calinski-Harabasz Index: 50378435.61918042
Percentage of similarity: 0.7983960689783052
Running cross-validation iteration 4
Silhouette Score: 0.7536448836326599
Davies-Bouldin Index: 0.06417127259147974
Calinski-Harabasz Index: 50378435.61918042
Percentage of similarity: 0.7983960689783052
Running cross-validation iteration 5
Silhouette Score: 0.7536448836326599
Davies-Bouldin Index: 0.06417127259147974
Calinski-Harabasz Index: 50378435.61918042
Percentage of similarity:

In [42]:
pca_kmeans_cv_df.to_csv("pca_kmeans_cv_df", index = False)
pca_agglomerative_cv_df.to_csv("pca_agglomerative_cv_df", index = False)
tsne_kmeans_cv_df.to_csv("tsne_kmeans_cv_df", index = False)
tsne_agglomerative_cv_df.to_csv("tsne_agglomerative_cv_df", index = False)